In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [2]:
import re

In [4]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt

In [ ]:
!pip install textblob
!pip install wordcloud

In [5]:
from string import punctuation
from sklearn import svm
from textblob import Word
from wordcloud import WordCloud
import textblob
from textblob import TextBlob

In [6]:
from imblearn.over_sampling import SMOTE

In [7]:
df = pd.read_csv("sample30.csv")

In [8]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [9]:
df['cleaned_reviews'] = df['reviews_text'].apply(lambda x: re.sub(r"http\S+", "", x))
df['cleaned_reviews']

0        i love this album. it's very good. more to the...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: cleaned_reviews, Length: 30000, dtype: object

In [10]:
def fix_contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"would't", "would not", s)
    s = re.sub(r"could't", "could not", s)
    s = re.sub(r"\'d", " would", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s

df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x:fix_contractions(x))
df['cleaned_reviews']

0        i love this album. it is very good. more to th...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: cleaned_reviews, Length: 30000, dtype: object

In [11]:
#Tokenize the review text to words
reg = re.compile('[^a-zA-Z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

df['cleaned_reviews'] = df['cleaned_reviews'].apply(tokenizeReview)
df['cleaned_reviews']

0        i love this album it is very good more to the ...
1        good flavor this review was collected as part ...
2                                              good flavor
3        i read through the reviews on here before look...
4        my husband bought this gel for us the gel caus...
                               ...                        
29995    i got this conditioner with influenster to try...
29996    i love it i received this for review purposes ...
29997    first of all i love the smell of this product ...
29998    i received this through influenster and will n...
29999    i received this product complimentary from inf...
Name: cleaned_reviews, Length: 30000, dtype: object

In [12]:
df[['reviews_text', 'cleaned_reviews']]

,reviews_text,cleaned_reviews
0,i love this album. it's very good. more to the...,i love this album it is very good more to the ...
1,Good flavor. This review was collected as part...,good flavor this review was collected as part ...
2,Good flavor.,good flavor
3,I read through the reviews on here before look...,i read through the reviews on here before look...
4,My husband bought this gel for us. The gel cau...,my husband bought this gel for us the gel caus...
...,...,...
29995,I got this conditioner with Influenster to try...,i got this conditioner with influenster to try...
29996,"I love it , I received this for review purpose...",i love it i received this for review purposes ...
29997,First of all I love the smell of this product....,first of all i love the smell of this product ...
29998,I received this through Influenster and will n...,i received this through influenster and will n...


In [13]:
df.shape

(30000, 16)

In [14]:
df = df.drop(['reviews_text'], axis=1)

In [15]:
df.shape

(30000, 15)

In [16]:
df['cleaned_reviews'].head(5)

0    i love this album it is very good more to the ...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: cleaned_reviews, dtype: object

In [17]:
df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [18]:
def senti(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = df['cleaned_reviews']
y = df['user_sentiment'].apply(senti)

In [19]:
print(len(X))
print(len(y))

30000
30000


In [20]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

In [21]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

In [22]:
#Indicates class imbalance
y.value_counts()

1    26632
0     3368
Name: user_sentiment, dtype: int64

In [23]:
#Handle class imbalance
X_features = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size = 0.2, random_state=43)
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)

In [24]:
def model_fit(X_train_smote, y_train_smote, X_test_smote, y_test_smote, ml_model, coef_show):
    clf = ml_model.fit(X_train_smote, y_train_smote)
    clf_pred = clf.predict(X_test_smote)
    accuracy = clf.score(X_test_smote, y_test_smote)
    model_performance = classification_report(y_test_smote, clf_pred)
    validation_pred_proba_grad = clf.predict_proba(X_test_smote)
    roc_auc = roc_auc_score(y_test_smote, validation_pred_proba_grad[:,1])
    
    print ('accuracy of the model: ', accuracy)
    print('')
    print(model_performance)
    print('')
    print('ROC_AUC score: ', roc_auc)
    
    print('-----------------------------')
    if coef_show == 1:
        w = tfidf.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('Top 10 positive features (variables)')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('Top 10 negative features (variables)')        
        print(coeff_df.tail(20).to_string(index=False))
    
    return clf

#### Logistic Regression Modeling:

In [25]:
log_reg = model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     LogisticRegression(solver='lbfgs', max_iter=500),
                                     coef_show=1)

accuracy of the model:  0.9382022471910112

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      5340
           1       0.95      0.93      0.94      5340

    accuracy                           0.94     10680
   macro avg       0.94      0.94      0.94     10680
weighted avg       0.94      0.94      0.94     10680


ROC_AUC score:  0.980863387058312
-----------------------------

Top 10 positive features (variables)
         Word  Coefficient
        great    18.529386
         love    15.381526
         best    12.156413
         easy    11.340542
        clean    10.575524
         good     9.868174
        loved     8.666248
       better     8.502735
    excellent     8.271085
         nice     8.189531
      perfect     7.536109
      awesome     7.507319
      enjoyed     7.243838
     favorite     7.194781
      amazing     7.025612
    wonderful     6.551349
         free     6.115241
 entertaining     5.943732
        handy

In [26]:
#Test some random text for classification - Do reviews classification as good or bad sentiment. 
Xfeatu = tfidf.transform(['this is a perfect fit', 
                          'amazing fragrance and awesome taste',
                          'an awful product',
                          'clean explanation handy parts',
                          'dirty appearance nasty odor'])
print(log_reg.predict(Xfeatu))

[1 1 0 1 0]


#### Multinomial Bayes

In [ ]:
bayes_model = model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     MultinomialNB(),
                                     coef_show=1)

In [29]:
Xfeatu = tfidf.transform(['this is a perfect fit', 
                          'amazing fragrance and awesome taste',
                          'an awful product',
                          'clean explanation handy parts',
                          'dirty appearance nasty odor'])
bayes_model.predict(Xfeatu)

array([1, 1, 0, 1, 0], dtype=int64)

#### Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
rand_forest = RandomForestClassifier(n_estimators=500, max_depth=20, max_features=500, random_state=100, oob_score=True)

In [32]:
random_forest_model = model_fit(X_train_smote, y_train_smote, 
                                                 X_test_smote, y_test_smote, 
                                                 rand_forest,
                                                 coef_show=0)

accuracy of the model:  0.8854868913857677

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      5340
           1       0.92      0.85      0.88      5340

    accuracy                           0.89     10680
   macro avg       0.89      0.89      0.89     10680
weighted avg       0.89      0.89      0.89     10680


ROC_AUC score:  0.9378377800221631
-----------------------------


In [33]:
Xfeatu = tfidf.transform(['this is a perfect fit', 
                          'amazing fragrance and awesome taste',
                          'an awful product',
                          'clean explanation handy parts',
                          'dirty appearance nasty odor'])
random_forest_model.predict(Xfeatu)

array([1, 1, 0, 1, 0], dtype=int64)

#### Recommendation System - User based collaborative

In [34]:
# Reading ratings file
ratings = pd.read_csv('sample30.csv' , encoding='latin-1')
ratings.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [35]:
ratings = ratings[['id', 'reviews_username', 'name', 'reviews_rating']]
ratings.head()

,id,reviews_username,name,reviews_rating
0,AV13O1A8GV-KLJ3akUyj,joshua,Pink Friday: Roman Reloaded Re-Up (w/dvd),5
1,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
2,AV14LG0R-jtxr-f38QfS,dorothy w,Lundberg Organic Cinnamon Toast Rice Cakes,5
3,AV16khLE-jtxr-f38VFn,rebecca,K-Y Love Sensuality Pleasure Gel,1
4,AV16khLE-jtxr-f38VFn,walker557,K-Y Love Sensuality Pleasure Gel,1


In [36]:
ratings.reviews_rating.unique()

array([5, 1, 3, 4, 2], dtype=int64)

In [37]:
ratings.reviews_rating.value_counts()

5    20831
4     6020
1     1384
3     1345
2      420
Name: reviews_rating, dtype: int64

In [38]:
print(ratings.shape)
print(ratings.id.value_counts())

(30000, 4)
AVpf3VOfilAPnD_xjpun    8545
AVpfPaoqLJeJML435Xk9    3325
AVpfJP1C1cnluZ0-e3Xy    2039
AVpfW8y_LJeJML437ySW    1186
AVpfRTh1ilAPnD_xYic2    1143
                        ... 
AVpe6_VI1cnluZ0-Z4do       1
AVpfDbSP1cnluZ0-c3wo       1
AVpfbpzd1cnluZ0-kqJV       1
AVpfifml1cnluZ0-mjSb       1
AVpfRxSkilAPnD_xYrzm       1
Name: id, Length: 271, dtype: int64


### Dividing the dataset into train and test

In [39]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.30, random_state=31)

In [40]:
print(train.shape)
print(test.shape)

(21000, 4)
(9000, 4)


In [41]:
print(len(ratings.reviews_username.unique()))
print(len(ratings.name.unique()))

tmp = ratings.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
tmp.head()

24915
271


name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Pivot the train ratings' dataset into matrix format in which columns are items and the rows are user.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)
df_pivot.head(5)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating dummy train & dummy test dataset
These dataset will be used for prediction 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction in dummy train dataset. 

- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [43]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,5
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,4
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,5
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,5


In [44]:
print(dummy_train['reviews_rating'].value_counts())
print(dummy_train.shape)

5    14514
4     4234
1      995
3      957
2      300
Name: reviews_rating, dtype: int64
(21000, 4)


In [45]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

In [46]:
dummy_train.reviews_rating.unique()

array([0], dtype=int64)

In [47]:
dummy_train.head()

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,0
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,0
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",0
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,0
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,0


In [48]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(1)

In [49]:
dummy_train.shape

(18252, 252)

#### Checking for a given user if the reviewed items are set to 0 and rest to 1. All good so far.

In [50]:
print(dummy_train.loc['abby'].value_counts())

1.0    249
0.0      3
Name: abby, dtype: int64


In [51]:
dummy_train

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01impala,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0325home,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxcsdfd,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
zxjki,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


**Cosine Similarity**

Cosine Similarity is a measurement that quantifies the similarity between two vectors [Which is Rating Vector in this case] 

**Adjusted Cosine**

Adjusted cosine similarity is a modified version of vector-based similarity where we incorporate the fact that different users have different ratings schemes. In other words, some users might rate items highly in general, and others might give items lower ratings as a preference. To handle this nature from rating given by user , we subtract average ratings for each user from each user's rating for different movies.

# User Similarity Matrix

## Using Cosine Similarity
#### Pair-wise similarity is done row-wise in the pivot table

In [52]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [53]:
user_correlation[np.isnan(user_correlation)] = 0

In [54]:
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [55]:
user_correlation.shape

(18252, 18252)

## Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean only for the movies rated by the user

In [56]:
# Create a user-item review matrix.
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [57]:
df_pivot.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalising the rating of the movie for each user around 0 mean

In [58]:
#This does the mean for each row as the axis is 1
mean = np.nanmean(df_pivot, axis=1)

In [59]:
print(mean)
print(len(mean))

[4. 1. 3. ... 2. 5. 5.]
18252


In [60]:
df_subtracted = (df_pivot.T - mean).T

In [61]:
df_subtracted.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0325home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06stidriver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finding cosine similarity

In [62]:
from sklearn.metrics.pairwise import pairwise_distances

In [63]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Prediction - User User

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [64]:
user_correlation[user_correlation<0]=0
user_correlation

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Rating predicted by the user (for items rated as well as not rated) is the weighted sum of correlation with the item rating (as present in the rating dataset). 

In [65]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
user_predicted_ratings.shape

(18252, 252)

Since we are interested only in the movies not rated by the user, we will ignore the movies rated by the user by making it zero. 

In [67]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
d = user_final_rating.loc['abby'].sort_values(ascending=False)[0:10]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd        6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)    5.679494
Clorox Disinfecting Bathroom Cleaner                                   3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                   3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                         3.370093
Tostitos Bite Size Tortilla Chips                                      3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total              3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo               3.370093
Chester's Cheese Flavored Puffcorn Snacks                              3.333333
Tostitos Simply Blue Corn Tortilla Chips                               3.333333
Name: abby, dtype: float64

In [69]:
# Take the user ID as input.
user_input = input("Enter your user name")
print(user_input)

Enter your user nameabby
abby


In [70]:
user_final_rating.head(2)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
d = user_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
d

name
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                                  6.269380
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                              5.679494
Clorox Disinfecting Bathroom Cleaner                                                             3.651637
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                                             3.370093
The Resident Evil Collection 5 Discs (blu-Ray)                                                   3.370093
Tostitos Bite Size Tortilla Chips                                                                3.370093
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                                        3.370093
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                                         3.370093
Chester's Cheese Flavored Puffcorn Snacks                                                        3.333333
Tostitos Simply Blue Corn Tortilla Chips 

### Evaluation - User User 

Evaluation will be same as you have seen above for the prediction. The only difference being, you will evaluate for the item already rated by the user insead of predicting it for the movie not rated by the user. 

In [72]:
# Find out the common users of test and train dataset.
common = test[test.reviews_username.isin(train.reviews_username)]
common.shape

(2049, 4)

In [73]:
common.head()

,id,reviews_username,name,reviews_rating
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1
15098,AVpf4oLxLJeJML43FcxC,arlene,Avery174 Ready Index Contemporary Table Of Con...,5
17968,AVpfcu821cnluZ0-k8ep,mimi3,Vaseline Intensive Care Lip Therapy Cocoa Butter,5
28020,AVpfRTh1ilAPnD_xYic2,movielover,Planes: Fire Rescue (2 Discs) (includes Digita...,5


In [74]:
# convert into the user-item matrix.
common_user_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [75]:
common_user_based_matrix.head()

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [77]:
user_correlation_df

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df_subtracted.head(1)

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_subtracted.index

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', name='reviews_username', length=18252)

In [80]:
user_correlation_df['reviews_username'] = df_subtracted.index
user_correlation_df.set_index('reviews_username',inplace=True)
user_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
common.head(1)

,id,reviews_username,name,reviews_rating
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5


In [82]:
list_name = common.reviews_username.tolist()
user_correlation_df.columns = df_subtracted.index.tolist()
user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [83]:
len(list_name)

2049

In [84]:
user_correlation_df.columns

Index(['00dog3', '00sab00', '01impala', '0325home', '06stidriver', '09mommy11',
       '1085', '10ten', '11111111aaaaaaaaaaaaaaaaa', '1143mom',
       ...
       'zozo0o', 'zpalma', 'zsazsa', 'zt313', 'zubb', 'zulaa118', 'zxcsdfd',
       'zxjki', 'zyiah4', 'zzdiane'],
      dtype='object', length=18252)

In [85]:
user_correlation_df_1

,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zapman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zburt5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
user_correlation_df_1.shape

(1728, 18252)

In [87]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [88]:
user_correlation_df_3 = user_correlation_df_2.T

In [89]:
user_correlation_df_3.head()

,00sab00,1234,1234asdf,123charlie,1943,4cloroxl,4djs,7inthenest,85lisa,a hutch,...,ygtz,yoadrian,yohnie1,yummy,yvonne,zach,zapman,zburt5,zebras,zmom
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234asdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123charlie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1943,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
user_correlation_df_3.shape

(1728, 1728)

In [91]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').fillna(0)

In [93]:
dummy_test.shape

(1728, 126)

In [94]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [95]:
common_user_predicted_ratings.head(2)

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Calculating the RMSE for only the movies rated by user. For RMSE, normalising the rating to (1,5) range.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [97]:
y.shape

(1728, 126)

In [98]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating')

In [99]:
common_

name,100:Complete First Season (blu-Ray),"42 Dual Drop Leaf Table with 2 Madrid Chairs""",Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"BRIDGESTONE 130/70ZR18M/C(63W)FRONT EXEDRA G851, CRUISER RADL",...,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00sab00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123charlie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zapman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
zburt5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [101]:
print(total_non_nan)

314


In [102]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

2.498437231297542


## Using Item similarity

# Item Based Similarity

Taking the transpose of the rating matrix to normalize the rating around the mean for different item. In the user based similarity, we had taken mean for each user instead of each item. 

In [103]:
df_pivot = train.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T
df_pivot.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalising the movie rating for each movie for using the Adujsted Cosine

In [104]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [105]:
df_subtracted.head()

reviews_username,00dog3,00sab00,01impala,0325home,06stidriver,09mommy11,1085,10ten,11111111aaaaaaaaaaaaaaaaa,1143mom,...,zozo0o,zpalma,zsazsa,zt313,zubb,zulaa118,zxcsdfd,zxjki,zyiah4,zzdiane
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100:Complete First Season (blu-Ray),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2x Ultra Era with Oxi Booster, 50fl oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finding the cosine similarity using pairwise distances approach

In [106]:
from sklearn.metrics.pairwise import pairwise_distances

In [107]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          1.          0.         ... -0.00657044  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.00657044  0.         ...  1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]


Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [108]:
item_correlation[item_correlation<0]=0
item_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Prediction - Item Item

In [109]:
item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)
item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.02352264, 0.        ,
        0.        ],
       [0.        , 0.00117139, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04848007, 0.        , ..., 0.        , 0.        ,
        0.00338607],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00062229, 0.00063779,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00155571, 0.00159447,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03252811, 0.02124867,
        0.        ]])

In [110]:
item_predicted_ratings.shape

(18252, 252)

In [111]:
dummy_train.shape

(18252, 252)

### Filtering the rating only for the movies not rated by the user for recommendation

In [112]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000


### Finding the top 5 recommendation for the *user*

In [113]:
# Take the user ID as input
user_input = input("Enter your user name")
print(user_input)

Enter your user nameabby
abby


In [114]:
# Recommending the Top 5 products to the user.
t20 = item_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
t20

name
Chester's Cheese Flavored Puffcorn Snacks                                         0.546107
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                             0.167279
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                          0.086751
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.042787
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               0.032291
Nexxus Exxtra Gel Style Creation Sculptor                                         0.030927
Aussie Aussome Volume Shampoo, 13.5 Oz                                            0.028226
Mike Dave Need Wedding Dates (dvd + Digital)                                      0.027196
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans                                   0.023710
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)                     0.021194
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks                                0.0

# Evaluation - Item Item

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [115]:
test.columns

Index(['id', 'reviews_username', 'name', 'reviews_rating'], dtype='object')

In [116]:
common =  test[test.name.isin(train.name)]
common.shape

(8978, 4)

In [117]:
common.head(4)

,id,reviews_username,name,reviews_rating
4332,AVpe9W4D1cnluZ0-avf0,sansname,Hoover174 Platinum Collection153 Lightweight B...,5
3434,AVpe5s3RLJeJML43x4eA,jennypstrong,Pantene Pro-V Expert Collection Age Defy Condi...,5
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1


In [118]:
common_item_based_matrix = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [119]:
common_item_based_matrix.shape

(199, 8371)

In [120]:
item_correlation_df = pd.DataFrame(item_correlation)

In [121]:
item_correlation_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
item_correlation_df['name'] = df_subtracted.index
item_correlation_df.set_index('name',inplace=True)
item_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"2x Ultra Era with Oxi Booster, 50fl oz",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
list_name = common.name.tolist()

In [124]:
item_correlation_df.columns = df_subtracted.index.tolist()
item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [125]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]
item_correlation_df_3 = item_correlation_df_2.T

In [126]:
item_correlation_df_3.head()

,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),4C Grated Parmesan Cheese 100% Natural 8oz Shaker,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",Annie's Homegrown Gluten Free Double Chocolate Chip Granola Bars,Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz",...,Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Wagan Smartac 80watt Inverter With Usb,"Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
name,,,,,,,,,,,,,,,,,,,,,
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100:Complete First Season (blu-Ray),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4C Grated Parmesan Cheese 100% Natural 8oz Shaker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Africa's Best No-Lye Dual Conditioning Relaxer System Super,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
item_correlation_df_3[item_correlation_df_3<0]=0

common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08080011, 0.06464009, ..., 0.01395515, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01951687, 0.        , 0.        , ..., 0.        , 0.00155571,
        0.00124457],
       [0.0127492 , 0.        , 0.        , ..., 0.        , 0.00159447,
        0.00127557],
       [0.        , 0.00564345, 0.00451476, ..., 0.        , 0.        ,
        0.        ]])

In [128]:
common_item_predicted_ratings.shape

(199, 8371)

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the items rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [129]:
dummy_test = common.copy()
dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)
dummy_test = dummy_test.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T.fillna(0)
common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.

In [130]:
common_ = common.pivot_table(index='reviews_username', columns='name', values='reviews_rating').T

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [132]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [133]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

3.559992328750575


In [134]:
train

,id,reviews_username,name,reviews_rating
11155,AVpf3VOfilAPnD_xjpun,gor05751,Clorox Disinfecting Wipes Value Pack Scented 1...,5
22114,AVpf--o9LJeJML43G_Fq,annavsl,Vaseline Intensive Care Healthy Hands Stronger...,4
26218,AVpfPkrkilAPnD_xX-L9,bluenose,"Finish Gelpacs Dishwasher Detergent, Orange Sc...",5
18966,AVpfJP1C1cnluZ0-e3Xy,kiki,Clorox Disinfecting Bathroom Cleaner,5
29999,AVpfW8y_LJeJML437ySW,kcoopxoxo,L'or233al Paris Elvive Extraordinary Clay Reba...,5
...,...,...,...,...
3735,AVpe7sl91cnluZ0-aI1Y,mary,Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans,5
14946,AVpf3VOfilAPnD_xjpun,mwah25,Clorox Disinfecting Wipes Value Pack Scented 1...,5
27518,AVpfRTh1ilAPnD_xYic2,lilym,Planes: Fire Rescue (2 Discs) (includes Digita...,4
26640,AVpfPPkEilAPnD_xX3cP,kim,Aveeno Baby Continuous Protection Lotion Sunsc...,5


In [135]:
test

,id,reviews_username,name,reviews_rating
4332,AVpe9W4D1cnluZ0-avf0,sansname,Hoover174 Platinum Collection153 Lightweight B...,5
3434,AVpe5s3RLJeJML43x4eA,jennypstrong,Pantene Pro-V Expert Collection Age Defy Condi...,5
20634,AVpfkak01cnluZ0-nJj6,girlie,Starbucks Iced Expresso Classics Vanilla Latte...,5
26479,AVpfPPkEilAPnD_xX3cP,kasheville,Aveeno Baby Continuous Protection Lotion Sunsc...,1
29726,AVpfW8y_LJeJML437ySW,payy,L'or233al Paris Elvive Extraordinary Clay Reba...,5
...,...,...,...,...
28375,AVpfrTyiLJeJML43BrSI,bala,Various Artists - Choo Choo Soul (cd),5
20889,AVpfM_ytilAPnD_xXIJb,ronald,Tostitos Bite Size Tortilla Chips,2
3505,AVpe6FfKilAPnD_xQmHi,elunsford,Chex Muddy Buddies Brownie Supreme Snack Mix,5
11361,AVpf3VOfilAPnD_xjpun,meli23melo,Clorox Disinfecting Wipes Value Pack Scented 1...,5


In [136]:
item_final_rating

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),"All,bran Complete Wheat Flakes, 18 Oz.",...,Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.023523,0.000000,0.000000
00sab00,0.0,0.001171,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
01impala,0.0,0.048480,0.0,0.0,0.0,0.0,0.0,0.0,0.014725,0.000000,...,0.0,0.0,0.0,0.006729,0.002859,0.0,0.0,0.000000,0.000000,0.003386
0325home,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
06stidriver,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,0.0,0.013955,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.039908,0.000000,0.0,0.0,0.000000,0.000000,0.000000
zxcsdfd,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.001334,0.004255,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001556,0.001594,0.000000
zxjki,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000534,0.001702,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000622,0.000638,0.000000


#### Identifying the sentiment of the review and picking top 5

In [137]:
df = pd.read_csv("sample30.csv")

#Do reviews classification as good or bad sentiment. 
tfidfFeatures = tfidf.transform(df.reviews_text)
sentClass = log_reg.predict(tfidfFeatures)

In [138]:
#Merge the class to the dataframe
sntmtClassSeries = pd.Series(sentClass, name = "sent_class")
df = df.join(sntmtClassSeries)
print(df[['manufacturer', 'name', 'reviews_text', 'sent_class']])

                             manufacturer  \
0      Universal Music Group / Cash Money   
1                                Lundberg   
2                                Lundberg   
3                                     K-Y   
4                                     K-Y   
...                                   ...   
29995                       L'oreal Paris   
29996                       L'oreal Paris   
29997                       L'oreal Paris   
29998                       L'oreal Paris   
29999                       L'oreal Paris   

                                                    name  \
0              Pink Friday: Roman Reloaded Re-Up (w/dvd)   
1             Lundberg Organic Cinnamon Toast Rice Cakes   
2             Lundberg Organic Cinnamon Toast Rice Cakes   
3                       K-Y Love Sensuality Pleasure Gel   
4                       K-Y Love Sensuality Pleasure Gel   
...                                                  ...   
29995  L'or233al Paris Elvive Extraordi

In [139]:
groupedDf = df.groupby(['name'])
product_class = groupedDf['sent_class'].agg(mean_class=np.mean)

In [140]:
print(type(product_class))
print(product_class.info)

<class 'pandas.core.frame.DataFrame'>
<bound method DataFrame.info of                                                     mean_class
name                                                          
0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. F...    0.500000
100:Complete First Season (blu-Ray)                   0.870504
2017-2018 Brownline174 Duraflex 14-Month Planne...    0.000000
2x Ultra Era with Oxi Booster, 50fl oz                1.000000
42 Dual Drop Leaf Table with 2 Madrid Chairs"         1.000000
...                                                        ...
Weleda Everon Lip Balm                                1.000000
Wilton Black Dots Standard Baking Cups                1.000000
Windex Original Glass Cleaner Refill 67.6oz (2 ...    0.715517
Yes To Carrots Nourishing Body Wash                   0.984615
Yes To Grapefruit Rejuvenating Body Wash              0.769231

[271 rows x 1 columns]>


In [141]:
product_class.index

Index(['0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest',
       '100:Complete First Season (blu-Ray)',
       '2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black',
       '2x Ultra Era with Oxi Booster, 50fl oz',
       '42 Dual Drop Leaf Table with 2 Madrid Chairs"',
       '4C Grated Parmesan Cheese 100% Natural 8oz Shaker',
       '5302050 15/16 FCT/HOSE ADAPTOR',
       'Africa's Best No-Lye Dual Conditioning Relaxer System Super',
       'Alberto VO5 Salon Series Smooth Plus Sleek Shampoo',
       'Alex Cross (dvdvideo)',
       ...
       'Walkers Stem Ginger Shortbread',
       'Wallmount Server Cabinet (450mm, 9 RU)',
       'Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee',
       'WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black',
       'Wedding Wishes Wedding Guest Book', 'Weleda Everon Lip Balm',
       'Wilton Black Dots Standard Baking Cups',
       'Windex Original 

In [142]:
for x in product_class.index:
    print(product_class.loc[x])

mean_class    0.5
Name: 0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest, dtype: float64
mean_class    0.870504
Name: 100:Complete First Season (blu-Ray), dtype: float64
mean_class    0.0
Name: 2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black, dtype: float64
mean_class    1.0
Name: 2x Ultra Era with Oxi Booster, 50fl oz, dtype: float64
mean_class    1.0
Name: 42 Dual Drop Leaf Table with 2 Madrid Chairs", dtype: float64
mean_class    0.833333
Name: 4C Grated Parmesan Cheese 100% Natural 8oz Shaker, dtype: float64
mean_class    0.0
Name: 5302050 15/16 FCT/HOSE ADAPTOR, dtype: float64
mean_class    0.833333
Name: Africa's Best No-Lye Dual Conditioning Relaxer System Super, dtype: float64
mean_class    0.666667
Name: Alberto VO5 Salon Series Smooth Plus Sleek Shampoo, dtype: float64
mean_class    0.801418
Name: Alex Cross (dvdvideo), dtype: float64
mean_class    0.454545
Name: All,bran Complete Wheat Flakes, 18 Oz., dtype: float64
mean_class    0.75
Name: Amb

Name: Hoover174 Platinum Collection153 Lightweight Bagged Upright Vacuum With Canister - Uh30010com, dtype: float64
mean_class    0.72449
Name: Hormel Chili, No Beans, dtype: float64
mean_class    1.0
Name: If A Man Answers (dvd), dtype: float64
mean_class    1.0
Name: Iman Luxury Moisturizing Lipstick, Black Brandy 006, dtype: float64
mean_class    0.933333
Name: Iman Second To None Stick Foundation, Clay 1, dtype: float64
mean_class    0.833333
Name: J.R. Watkins Hand Cream, Lemon Cream, dtype: float64
mean_class    1.0
Name: J.a. Henckels International 4-Pc Everedge Plus Steak Knife Set, dtype: float64
mean_class    1.0
Name: JNH Lifestyles Goldstar 3 Person FAR Infrared Sauna, dtype: float64
mean_class    0.828431
Name: Jason Aldean - They Don't Know, dtype: float64
mean_class    1.0
Name: Jergens Extra Moisturizing Liquid Hand Wash, 7.5oz, dtype: float64
mean_class    0.785714
Name: Johnson's Baby Bubble Bath and Wash, 15oz, dtype: float64
mean_class    0.962963
Name: Jolly Time S

Name: Sopranos:Season 6 Part 1 (blu-Ray), dtype: float64
mean_class    0.0
Name: Southern Enterprises Archer Fold-Away Home Bar in Walnut, dtype: float64
mean_class    0.625
Name: Spam Single Classic, dtype: float64
mean_class    0.5
Name: Stacy's Garden Veggie Medley Pita Chips, dtype: float64
mean_class    0.791667
Name: Stacy's Simply Naked Bagel Chips, dtype: float64
mean_class    0.538462
Name: Stander Pt Bedcane - Bed Handle, dtype: float64
mean_class    1.0
Name: Starbucks Iced Expresso Classics Vanilla Latte Coffee Beverage - 40oz, dtype: float64
mean_class    0.666667
Name: Starbucks153 Doubleshot Protein Coffee - 11 Fl Oz, dtype: float64
mean_class    0.930108
Name: Stargate (ws) (ultimate Edition) (director's Cut) (dvdvideo), dtype: float64
mean_class    0.0
Name: Stonyfield Yobaby Peach & Pear Yogurt 4oz 6 Ct, dtype: float64
mean_class    0.905303
Name: Storkcraft Tuscany Glider and Ottoman, Beige Cushions, Espresso Finish, dtype: float64
mean_class    0.8
Name: Suave Profe

In [144]:
# Take the item name as input
item_name = input("Item name please...").strip()
print(item_name)
print("Sentiment for the product is - ", product_class.loc[item_name][0])
type(product_class.loc[item_name])

Item name please...2x Ultra Era with Oxi Booster, 50fl oz
2x Ultra Era with Oxi Booster, 50fl oz
Sentiment for the product is -  1.0


pandas.core.series.Series

In [145]:
groupedDf.get_group('Craft Punch Giga Scallop Circle 45 24687534 To 334')

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment,sent_class
1262,AV1YpiJvvKc47QAVguxy,No Brand,"Home,Arts & Crafts,Scrapbooking,Scrapbooking T...",120,Craft Punch Giga Scallop Circle 45 24687534 To...,2017-05-08T17:54:36.000Z,NaN,True,5,Very easy on the hands to use. I used it on ca...,Great Punch,NaN,NaN,j9,Positive,1


In [146]:
t20

name
Chester's Cheese Flavored Puffcorn Snacks                                         0.546107
Windex Original Glass Cleaner Refill 67.6oz (2 Liter)                             0.167279
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                          0.086751
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.042787
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               0.032291
Nexxus Exxtra Gel Style Creation Sculptor                                         0.030927
Aussie Aussome Volume Shampoo, 13.5 Oz                                            0.028226
Mike Dave Need Wedding Dates (dvd + Digital)                                      0.027196
Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans                                   0.023710
Dark Shadows (includes Digital Copy) (ultraviolet) (dvdvideo)                     0.021194
Cheetos Crunchy Flamin' Hot Cheese Flavored Snacks                                0.0

In [147]:
for itmName in list(t20.index):
    t20[itmName] = product_class.loc[itmName][0]

In [148]:
t20.sort_values(ascending=False)[:5]

name
Yes To Carrots Nourishing Body Wash                                               0.984615
Pleasant Hearth Diamond Fireplace Screen - Espresso                               0.935484
Stargate (ws) (ultimate Edition) (director's Cut) (dvdvideo)                      0.930108
Red (special Edition) (dvdvideo)                                                  0.906250
Pleasant Hearth 1,800 sq ft Wood Burning Stove with Blower, Medium, LWS-127201    0.905660
Name: abby, dtype: float64